## Training dataset balancing
This notebook usig SMOTE library to balance training dataset

In [1]:
# import sys
# !{sys.executable} -m pip install category_encoders --user

In [2]:
import os
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import numpy as np
from matplotlib import pyplot
import collections
import category_encoders as ce
from lib.data_prepare import data_prepare

### 1. Read input file

In [3]:
df = pd.read_pickle("../3_dataset_split/training.pkl")
del df['id']
del df['sample_id']
del df['study_id']
del df['version']
df

,biome,exptype,GO:0043130,GO:0055074,GO:0055117,GO:0046933,GO:0006302,GO:0008643,GO:0043752,GO:0005976,...,GO:0016684,GO:0004650,GO:0030178,GO:0012511,GO:0030168,GO:0043657,GO:0016879,GO:0016977,GO:0006898,GO:0019357
16571,root:Host-associated:Human:Digestive system:La...,assembly,-0.189496,-0.051936,-0.075688,-0.330619,0.002727,-0.166729,-0.309479,-0.191756,...,-0.296579,-0.285802,-0.148909,-0.048156,-0.111776,-0.054362,-0.306251,-0.031340,-0.066268,-0.525327
24277,root:Mixed:Sediment:Sediment:Sediment,metagenomic,0.565269,-0.051936,-0.075688,0.027486,-0.248971,-0.177289,-0.146125,0.052316,...,-0.123640,-0.210765,0.119466,-0.048156,-0.111776,-0.054362,-0.128198,0.056683,-0.066268,-0.525327
1361,root:Environmental:Aquatic:Marine:Oceanic,metagenomic,-0.030598,-0.051936,-0.075688,2.998055,-0.186047,-0.022852,2.012133,3.641210,...,-0.247168,-0.122327,0.924590,-0.048156,-0.111776,-0.054362,0.427392,-0.031340,-0.066268,-0.525327
61642,root:Host-associated:Human:Skin,assembly,-0.189496,-0.051936,-0.075688,-0.337209,0.002727,-0.175969,-0.306179,-0.185108,...,-0.288344,-0.291162,-0.148909,-0.048156,-0.111776,-0.054362,-0.311197,-0.031340,-0.066268,1.688806
55105,root:Environmental:Aquatic:Marine,assembly,0.128300,-0.051936,-0.075688,-0.338308,0.254425,-0.154849,-0.319380,-0.191756,...,-0.321284,-0.299202,-0.014722,-0.048156,-0.111776,-0.054362,-0.332629,-0.031340,-0.066268,-0.525327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72858,root:Host-associated:Human:Digestive system:La...,assembly,-0.189496,-0.051936,-0.075688,-0.324851,0.317349,-0.168049,-0.292979,-0.190806,...,-0.255403,-0.245604,-0.148909,-0.048156,-0.111776,-0.054362,-0.299657,-0.031340,-0.066268,0.028206
61154,root:Host-associated:Human:Skin,assembly,-0.110047,-0.051936,-0.075688,-0.347370,-0.123122,-0.179929,-0.316080,-0.194605,...,-0.313049,-0.301882,-0.148909,-0.048156,-0.111776,-0.054362,-0.332629,-0.031340,-0.066268,-0.525327
24711,root:Mixed:Sediment:Sediment:Sediment,metagenomic,1.518657,-0.051936,-0.075688,0.516585,-0.186047,-0.109970,0.327438,0.385658,...,-0.065994,-0.031210,1.461339,-0.048156,-0.111776,-0.054362,0.260880,0.182432,-0.066268,-0.525327
1464,root:Environmental:Aquatic:Estuary:Sediment,assembly,-0.189496,-0.051936,-0.075688,-0.323753,-0.186047,-0.179929,-0.316080,-0.178460,...,-0.288344,-0.299202,-0.148909,-0.048156,-0.111776,-0.054362,-0.294711,-0.031340,-0.066268,-0.248561


### 2. Encode categarical features

In [4]:
encoder= ce.OrdinalEncoder(mapping=[
{'col': 'exptype', 'mapping': {'assembly': 0, 'metagenomic': 1}}
])
df = encoder.fit_transform(df)
df

,biome,exptype,GO:0043130,GO:0055074,GO:0055117,GO:0046933,GO:0006302,GO:0008643,GO:0043752,GO:0005976,...,GO:0016684,GO:0004650,GO:0030178,GO:0012511,GO:0030168,GO:0043657,GO:0016879,GO:0016977,GO:0006898,GO:0019357
16571,root:Host-associated:Human:Digestive system:La...,0,-0.189496,-0.051936,-0.075688,-0.330619,0.002727,-0.166729,-0.309479,-0.191756,...,-0.296579,-0.285802,-0.148909,-0.048156,-0.111776,-0.054362,-0.306251,-0.031340,-0.066268,-0.525327
24277,root:Mixed:Sediment:Sediment:Sediment,1,0.565269,-0.051936,-0.075688,0.027486,-0.248971,-0.177289,-0.146125,0.052316,...,-0.123640,-0.210765,0.119466,-0.048156,-0.111776,-0.054362,-0.128198,0.056683,-0.066268,-0.525327
1361,root:Environmental:Aquatic:Marine:Oceanic,1,-0.030598,-0.051936,-0.075688,2.998055,-0.186047,-0.022852,2.012133,3.641210,...,-0.247168,-0.122327,0.924590,-0.048156,-0.111776,-0.054362,0.427392,-0.031340,-0.066268,-0.525327
61642,root:Host-associated:Human:Skin,0,-0.189496,-0.051936,-0.075688,-0.337209,0.002727,-0.175969,-0.306179,-0.185108,...,-0.288344,-0.291162,-0.148909,-0.048156,-0.111776,-0.054362,-0.311197,-0.031340,-0.066268,1.688806
55105,root:Environmental:Aquatic:Marine,0,0.128300,-0.051936,-0.075688,-0.338308,0.254425,-0.154849,-0.319380,-0.191756,...,-0.321284,-0.299202,-0.014722,-0.048156,-0.111776,-0.054362,-0.332629,-0.031340,-0.066268,-0.525327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72858,root:Host-associated:Human:Digestive system:La...,0,-0.189496,-0.051936,-0.075688,-0.324851,0.317349,-0.168049,-0.292979,-0.190806,...,-0.255403,-0.245604,-0.148909,-0.048156,-0.111776,-0.054362,-0.299657,-0.031340,-0.066268,0.028206
61154,root:Host-associated:Human:Skin,0,-0.110047,-0.051936,-0.075688,-0.347370,-0.123122,-0.179929,-0.316080,-0.194605,...,-0.313049,-0.301882,-0.148909,-0.048156,-0.111776,-0.054362,-0.332629,-0.031340,-0.066268,-0.525327
24711,root:Mixed:Sediment:Sediment:Sediment,1,1.518657,-0.051936,-0.075688,0.516585,-0.186047,-0.109970,0.327438,0.385658,...,-0.065994,-0.031210,1.461339,-0.048156,-0.111776,-0.054362,0.260880,0.182432,-0.066268,-0.525327
1464,root:Environmental:Aquatic:Estuary:Sediment,0,-0.189496,-0.051936,-0.075688,-0.323753,-0.186047,-0.179929,-0.316080,-0.178460,...,-0.288344,-0.299202,-0.148909,-0.048156,-0.111776,-0.054362,-0.294711,-0.031340,-0.066268,-0.248561


In [5]:
new_df = data_prepare.dataset_balancing(df, 900)

In [6]:
collections.Counter(new_df['biome']).most_common()

[('root:Engineered:Bioreactor', 900),
 ('root:Engineered:Bioremediation:Terephthalate:Wastewater', 900),
 ('root:Engineered:Built environment', 900),
 ('root:Engineered:Food production', 900),
 ('root:Engineered:Food production:Dairy products', 900),
 ('root:Engineered:Food production:Fermented beverages', 900),
 ('root:Engineered:Modeled:Simulated communities (microbial mixture)', 900),
 ('root:Engineered:Solid waste:Composting', 900),
 ('root:Engineered:Wastewater', 900),
 ('root:Engineered:Wastewater:Activated Sludge', 900),
 ('root:Engineered:Wastewater:Water and sludge', 900),
 ('root:Environmental:Aquatic:Aquaculture', 900),
 ('root:Environmental:Aquatic:Estuary', 900),
 ('root:Environmental:Aquatic:Estuary:Sediment', 900),
 ('root:Environmental:Aquatic:Freshwater', 900),
 ('root:Environmental:Aquatic:Freshwater:Lake', 900),
 ('root:Environmental:Aquatic:Freshwater:Lentic', 900),
 ('root:Environmental:Aquatic:Freshwater:Lotic:Sediment', 900),
 ('root:Environmental:Aquatic:Freshwa

In [ ]:
new_df.to_pickle('balanced_train.pkl')
new_df.to_csv('balanced_train.tsv', sep="\t", index=False)